## Package Imports

In [1]:
# Import packages

import pandas as pd 
import numpy as np
import re
import nltk
from nltk import word_tokenize as w_tokenize
from nltk.stem import WordNetLemmatizer
lemma = WordNetLemmatizer()
from nltk.corpus import stopwords
import string
punc = string.punctuation
stop_words = set(stopwords.words('english') + ["..."])
from sklearn.feature_extraction.text import TfidfVectorizer as tfv
from sklearn.decomposition import NMF as nmf
import pdfminer
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfpage import PDFTextExtractionNotAllowed
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfdevice import PDFDevice
from pdfminer.high_level import extract_text
import PyPDF2
# !pip install PyPDF2

## Extract Text

In [2]:
## AS
# extract text from list of pdf titles and create dataframe of text
path = './Scripts/'
end = '.pdf'
titles = ['1917', '20th_Century_Women', 'BridgeofSpies', 'ExMachina', 'First_Reformed',
         'Get_Out', 'GreenBook', 'Hell_or_HighWater', 'InsideOut', 'KnivesOut', 'LadyBird',
         'LaLaLand', 'Manchester_By_TheSea', 'MarriageStory', 'Parasite', 'Roma', 
          'ShapeofWater', 'Spotlight', 'StraightOuttaCompton', 'TheBigSick', 
         'TheFavourite', 'TheLobster', 'ThreeBillboards', 'Vice']

movie_scripts = []
for title in titles:
    script = extract_text(path + title + end)
    movie_scripts.append(script)
    print (title)

# create dataframe of scripts/titles from lists    
movie_df = pd.DataFrame(list(zip(titles, movie_scripts)), columns = ['Title', 'Script'])

1917
20th_Century_Women
BridgeofSpies
ExMachina
First_Reformed
Get_Out
GreenBook
Hell_or_HighWater
InsideOut
KnivesOut
LadyBird
LaLaLand
Manchester_By_TheSea
MarriageStory
Parasite
Roma
ShapeofWater
Spotlight
StraightOuttaCompton
TheBigSick
TheFavourite
TheLobster
ThreeBillboards
Vice


In [3]:
## AR
# add label for original screenplay win, best picture win, and year
orig_screen_win = [0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
best_pic_win = ['Nom', None, 'Nom', None, None, 'Nom', 'Win', 'Nom', None, None, 'Nom', 'Nom', 'Nom', 'Nom', 'Win', 'Nom', 'Win', 'Win', None, None, 'Nom', None, 'Nom', 'Nom']
year = [2019, 2016, 2015, 2015, 2018, 2017, 2018, 2016, 2015, 2019, 2017, 2016, 2016, 2019, 2019, 2018, 2017, 2015, 2015, 2017, 2018, 2016, 2017, 2018]
reviewer = ["Shafer", "Shafer", "Shafer", "Shafer", "Ragan", "Shafer", "Ragan", "Ragan", "Shafer", "Shafer", "Ragan", "Ragan", "Shafer", "Ragan", "Shafer", "Ragan", "Ragan", "Shafer", "Shafer", "Ragan", "Ragan", "Shafer", "Ragan", "Ragan"]

# sanity check -- should return True
print(f"\n\nSanity check for lists: {len(orig_screen_win) == len(year) == len(best_pic_win) ==  len(list(movie_df['Title'])) == len(reviewer)}")

movie_df['year'] = year
movie_df['orig_screen_win'] = orig_screen_win
movie_df['best_pic_win'] = best_pic_win
movie_df['reviewer'] = reviewer



Sanity check for lists: True


## Data Cleaning

### AR

In [4]:
ar = movie_df[movie_df['reviewer'] == "Ragan"].copy()
raw_scripts = list(ar['Script'])

In [5]:
def drop_start(idx, todrop):
    return raw_scripts[idx].replace(todrop, "")

extraneous = ["Scripts.com\n\nFirst Reformed\n\nBy Paul Schrader\n\nPage 1/57\n\n\x0c", 
              "GREEN BOOK \n\nWritten by \n\nNick Vallelonga & Brian Currie & Peter Farrelly \n\n\x0c", 
              "Scripts.com\n\nHell or High Water\n\nBy Taylor Sheridan\n\nPage 1/42\n\n\x0c", 
              " \n \n\n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n\n \n \n \n \n \n \n \n \n \n \n \n\nLADY BIRD \n\nwritten by \n\nGreta Gerwig \n\n\x0c \n \n\n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n\nii. \n\n", 
              "LA LA LAND\n\nby\n\nDamien Chazelle\n\n\x0c", 
              "MARRIAGE STORY\n\nWritten and Directed by Noah Baumbach\n\n\x0cBlack.\n\n", 
              "IN ENGLISH\n\n\x0cROMA\n\nWritten and Directed by\n\nAlfonso Cuarón\n\nDates in RED are meant only as a tool for the different departments for \nthe specific historical accuracy of the scenes and are not intended to \nappear on screen. \n\n\x0cThursday, September 3rd, 1970\n\n", 
              "F O R   Y O U R   C O N S I D E R A T I O N\n\nB E S T   O R I G I N A L   S C R E E N P L A Y\n\nG U I L L E R M O   D E L   T O R O\n&\nV A N E S S A   T A Y L O R\n\n\x0c1                             \n\n1\n\n", 
              "THE BIG SICK\n\nby\nEmily V. Gordon & Kumail Nanjiani\n\n\x0c1\n\n", 
              "F O R   Y O U R   C O N S I D E R A T I O N\n\nW R I T T E N   B Y\n\nD E B O R A H   D A V I S   A N D   T O N Y   M C N A M A R A\n\n\x0cW R I T T E N   B Y\n\nD E B O R A H   D A V I S   A N D   T O N Y   M C N A M A R A\n\n\x0cTHE FAVOURITE\n\nWritten by\n\nDeborah Davis and Tony McNamara\n\nFINAL SHOOTING SCRIPT - 23rd MARCH 2017\n\nElement Pictures\n21 Mespil Road \nDublin 4\nIreland\n\nScarlet Films\n3 Oakley Studios\nUpper Cheyne Row\nLondon SW3 5JW, UK\n\n \n\n\x0c1\n\n2\n\n3\n\n4\n\n5\n\n6\n\n", 
              "F O R   Y O U R   C O N S I D E R A T I O N\n\nB E S T   O R I G I N A L   S C R E E N P L A Y\n\nM A R T I N   M c D O N A G H\n\n\x0cTHREE BILLBOARDS OUTSIDE EBBING, MISSOURI\n\nYou Red Welby?\n\nby\nMartin McDonagh\n\n1\n\n", 
              "Written and Directed by\n\nVICE\n\nAdam McKay\n\n\x0c"]

for r in range(len(raw_scripts)):
    raw_scripts[r] = drop_start(r, extraneous[r])

In [6]:
def expand_contractions(text):
    text = re.sub("\'s", " is", text) # it's --> it is
    text = re.sub("ain\'t", "are not", text) # ain't --> are not
    text = re.sub("n\'t", " not", text) # don't --> do not
    text = re.sub("\'re", " are", text) # you're --> you are
    text = re.sub("\'d", " would", text) # she'd --> she would
    text = re.sub("\'ll", " will", text) # he'll --> he will
    text = re.sub("\'ve", " have", text) # we've --> we have
    text = re.sub("\'m", " am", text) # I'm --> I am
    return text

for i in range(len(raw_scripts)):
    raw_scripts[i] = expand_contractions(raw_scripts[i])

In [7]:
def clean_lists(lis):
    cust_punct = string.punctuation + "`" + "'" + "’" + "”" + "“" + "..."
    if type(lis) != list:
        lis = w_tokenize(lis)
    lis = [l.replace("'", "") for l in lis]
    # strip punctuation
    lis = [t.lower() for t in lis if t not in set(cust_punct)]
    # lemmatize + lowercase
    lis = [lemma.lemmatize(l) for l in lis]
    # drop stop words
    lis = [t for t in lis if t not in set(stop_words)]
    # strip new lines
    lis = [re.sub('\s+', ' ', l) for l in lis]
    # remove all non-alphanumeric -- fail safe code
    lis = [re.sub("[^0-9a-zA-Z]+", "", l) for l in lis]
    return lis

In [8]:
def split_dialogue_everything(script, pages):
    first_ref = False
    hhw = False
    other = False

    if script == raw_scripts[0]:
        first_ref = True
        print("First Reformed")
    elif script == raw_scripts[2]:
        hhw = True
        print("Hell or High Water")
    else:
        other = True
        print("Other")
    script = script.replace("'", "")
    # remove page numbers
    script = re.sub(pages, " ", script)
    # remove voiceover --(V.O.)--from script
    script = script.replace("(V.O.)", "")
    # extract dialogue -- some require customization
    # First Reformed
    if first_ref:
        split_script = w_tokenize(script)
        orig_split_script = split_script.copy()
        # find indexes with (
        open_pars = [i for i, e in enumerate(split_script) if e == "("]
        # find indexes with )
        end_pars = [i for i, e in enumerate(split_script) if e == ")"]
        # return words between ( and ) to extract visual cues
        visual_cues_fr = [" ".join(split_script[open_pars[o]+1:end_pars[o]]) for o in range(len(open_pars))]
        # drop visual cues from list
        for o in range(len(open_pars)):
            del split_script[open_pars[o]:end_pars[0]+1]
        # re-join : with character names and drop extra :
        # find indices with :
        char_idx = [i for i, item in enumerate(split_script) if item.endswith(':')]
        # for every index with :, go to the index before and add : to re-join CHARACTER with :
        for b in char_idx:
            split_script[b-1] = split_script[b-1] + split_script[b]
        # drop extraneous colons
        if ":" in split_script:
            split_script.remove(":")
        # append non-character names to dialogue list
        dialogue = [split_script[t] for t in range(len(split_script)) if split_script[t].endswith(":") == False]
        script = orig_split_script
    # Hell or High Water -- script is literally just dialogue
    elif hhw:
        dialogue = script
    # All others
    else:
        # split on combo of caps and new lines
        split_script = re.split("\\n[A-Z]*\s?\\n", script)
        visual_cues = []
        # if begins with INT or EXT then it's a setting and the next line is visual cue
        settings = [v for v in range(len(split_script)) if split_script[v].startswith("INT")
                    or split_script[v].startswith("EXT")
                    or split_script[v].startswith("VISUALS|VISUAL")]
        # extract visual cues
        visual_cue = [split_script[s+1] for s in settings]
        # remove word if uppercase
        visual_cue = [w for w in visual_cue if w.isupper() == False]
        visual_cues.append(visual_cue)
        sets_cues_id = []
        for s in settings:
            sets_cues_id.append(s)
            sets_cues_id.append(s+1)
        # remove settings and visual cues from dialogue
        dialogue_ms = [split_script[d] for d in range(len(split_script)) if d not in sets_cues_id]
        # removing entries that are just all caps (i.e. CHARLIE)
        dialogue_wo_upper = [d for d in dialogue_ms if d.isupper() == False]
        # removing entries that don't have a character indicator
        dialogue_wo_indicator = [t for t in dialogue_wo_upper if t[:3].isupper() == False]
        # split out indicators from dialogue + flatten nested list
        dialogue_split = [re.split("[A-Z]\s?\\n", t) for t in dialogue_wo_upper if t[:3].isupper()]
        # drop indicator + flatten split list
        dialogue_flatten = []
        for ts in dialogue_split:
            ts.pop(0)
            dialogue_flatten.append(ts)
        dialogue_flatten = [item for sublist in dialogue_flatten for item in sublist]
        dialogue = dialogue_flatten + dialogue_wo_indicator
        dialogue = w_tokenize(" ".join(dialogue))
    # clean + join lists
    everything_list = " ".join(clean_lists(script))
    dialogue_list = " ".join(clean_lists(dialogue))
    # return everything list and dialogue list
    return everything_list, dialogue_list

In [9]:
# regex codes for page numbers
page = ["\\nPage \d+/57",
       "\\x0c                                   \\n\\n   \d+\.",
       "\\nPage \d+/42",
       "\\x0c\s?[\\n\\n ?\\n ?]+\d+\. ?\\n",
       "Revision\\n\\n\d+\.\\n",
       "\\x0c                                                 \d+\.\\n",
       "\\x0c\d+\.\\n",
       "\\x0c\d+                             \\n\\n\d+",
       "\\n\\x0c\d+\\n",
       "\\x0cTHE FAVOURITE SHOOTING SCRIPT 6 MAR 17   \d+.\\n",
       "\\n\d+\.",
       "\\x0c\d+\.\\n"]

In [10]:
# custom pre-processing code
# La La Land 
raw_scripts[4] = re.sub("\\n\d+\\n", "", raw_scripts[4])
raw_scripts[4] = re.sub("\\n[A-Z]\d+\\n", "", raw_scripts[4])
raw_scripts[4] = re.sub("\[(.*?)\]", "", raw_scripts[4])

# Shape of Water 
raw_scripts[7] = raw_scripts[7].replace("\n\nTHE SHAPE OF WATER\nTHE SHAPE OF WATER\nWritten by\n\nGuillermo del Toro\nWritten by\n&\nVanessa Taylor\nGuillermo del Toro\n&\nVanessa Taylor\nStory by\nGuillermo del Toro\n\nStory by\nGuillermo del Toro\n\nFOX SEARCHLIGHT PICTURES, INC. \n10201 W. Pico Blvd.\nLos Angeles, CA 90035\n\nFOX SEARCHLIGHT PICTURES, INC. \n10201 W. Pico Blvd.\nLos Angeles, CA 90035\n\nALL RIGHTS RESERVED. COPYRIGHT ©2016 WILLOW AND OAK, INC. NO PORTION OF THIS SCRIPT MAY BE \nPERFORMED, PUBLISHED, REPRODUCED, SOLD OR DISTRIBUTED BY ANY MEANS, OR QUOTED OR PUBLISHED IN ANY \nMEDIUM, INCLUDING ANY WEB SITE, WITHOUT THE PRIOR WRITTEN CONSENT OF WILLOW AND OAK, INC. DISPOSAL \nOF THIS SCRIPT COPY DOES NOT ALTER ANY OF THE RESTRICTIONS SET FORTH ABOVE.\n\nALL RIGHTS RESERVED. COPYRIGHT ©2016 WILLOW AND OAK, INC. NO PORTION OF THIS SCRIPT MAY BE \nPERFORMED, PUBLISHED, REPRODUCED, SOLD OR DISTRIBUTED BY ANY MEANS, OR QUOTED OR PUBLISHED IN ANY \nMEDIUM, INCLUDING ANY WEB SITE, WITHOUT THE PRIOR WRITTEN CONSENT OF WILLOW AND OAK, INC. DISPOSAL \nOF THIS SCRIPT COPY DOES NOT ALTER ANY OF THE RESTRICTIONS SET FORTH ABOVE.", "")
raw_scripts[7] = re.sub("\\n\d+\\n", "", raw_scripts[7])
raw_scripts[7] = re.sub("\\n[A-Z]\d+\\n", "", raw_scripts[7])

# The Favourite 
raw_scripts[9] = re.sub("\\n\d+\\n", "", raw_scripts[9])

# Three Billboards 
raw_scripts[10] = re.sub("\\n\d+\\n", "", raw_scripts[10])

In [11]:
# loop over all scripts and apply function to split dialogue from everything + clean
# instantiate empty lists to become columns
master_dialogue = []
master_cleaned_everything = []
for r in range(len(raw_scripts)):
    # apply function to every script
    everything, dialogue = split_dialogue_everything(raw_scripts[r], page[r])
    # add to master lists
    master_cleaned_everything.append(everything)
    master_dialogue.append(dialogue)

# sanity check
len(master_dialogue) == len(master_cleaned_everything)

First Reformed
Other
Hell or High Water
Other
Other
Other
Other
Other
Other
Other
Other
Other


True

In [12]:
# return processed scripts to data frame
# sanity checks -- Expect 3 returns of True
len(master_dialogue) == 12
len(master_cleaned_everything) == 12
len(master_dialogue) == len(master_cleaned_everything)

# make column of dialogue
ar['cleaned_dialogue'] = master_dialogue

# make everything column
ar['cleaned_script'] = master_cleaned_everything

In [13]:
ar.head()

,Title,Script,year,orig_screen_win,best_pic_win,reviewer,cleaned_dialogue,cleaned_script
4,First_Reformed,Scripts.com\n\nFirst Reformed\n\nBy Paul Schra...,2018,0,None,Ragan,keep journal word program digital file longhan...,bird calling distance toller keep journal word...
6,GreenBook,GREEN BOOK \n\nWritten by \n\nNick Vallelonga ...,2018,1,Win,Ragan,thank much welcome jules podell copacabana bob...,fade title card new york city 1962 ext copacab...
7,Hell_or_HighWater,Scripts.com\n\nHell or High Water\n\nBy Taylor...,2016,0,Nom,Ragan,quiet open door open door devil please stand t...,quiet open door open door devil please stand t...
10,LadyBird,\n \n\n \n \n \n \n \n \n \n \n \n \n \n \n \...,2017,0,Nom,Ragan,making bed looking back loan scholarship world...,anybody talk california hedonism ha never spe...
11,LaLaLand,LA LA LAND\n\nby\n\nDamien Chazelle\n\n A sun-...,2016,0,Nom,Ragan,think going worst working like gon na talk tal...,sunblasted sky hear radio one piece music ano...


### AS

In [14]:
ACS = movie_df[movie_df['reviewer'] == 'Shafer'].copy()

In [15]:
# Create function to remove extraneous information from title pages, etc.

ext_list = ['1917\n\nWritten by\n\nSam Mendes\n&\nKrysty Wilson-Cairns\n\n© 2018 Storyworks Productions Limited. All rights reserved.\n\n\x0cii\nii\n\n‘Life, to be sure,\n Is nothing much to lose,\n But young men think it is,\n And we were young.’\n\n-A.E. Housman\n\n‘We have so much to say, and we shall never say it.’\n\n-Erich Maria Remarque\nAll Quiet On The Western Front \n\n\x0ciii\niii\n\nThe following script takes place in real time, and - with the \nexception of one moment - is written and designed to be one single \ncontinuous shot. \n\nNOTE:\n\n\x0c                                                                1.\n',
             '20th Century Women\n\nby\n\nMike Mills\n\n\x0cEXT.',
             'Scripts.com\n\nBridge of Spies\n\nBy Matt Charman\n\nPage 1/124\n\n\x0cTITLE OVER BLACK: 1957.',
             ' \n\nEX MACHINA\n\nBy','\n\nAlex Garland\n\n','Alex Garland 2013\nc/o DNA Films Ltd.',
            '10 Amwell Street\nLondon EC1R 1UQ\nT:','\n\+44 (0)207 843 4410\n\n\x0c','\n\n\x0c'
             'Scripts.com\n\nGet Out\n\nBy Jordan Peele\n\nPage 1/94\n\n\x0c',
             'INSIDE OUT\n\noriginal story by\n\nPete Docter\nRonnie Del Carmen\n \n\nScreenplay by\n\nPete Docter\nMeg LeFauve\nJosh Cooley\n\n\x0cBLACK.',
             'KNIVES OUT\n\nA Murder Mystery by\n\nRian Johnson\n\nSCREEN SCRIPT\n\n\x0cEXT.',
             'MANCHESTER BY THE SEA\n\nWritten & Directed \n\nby\n\nKenneth Lonergan\n\n\x0cEXT.',
             'F O R   Y O U R   C O N S I D E R A T I O N\n\nOUTSTANDING ORIGINAL SCREENPLAY \nSCREENPLAY BY\nBONG JOON HO AND HAN JIN WON\nSTORY BY\nBONG JOON HO\n\n\x0c \n \n \n \n \n \n \n \n \n\n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n\n \n \nPARASITE\n \n \n \n\nScreenplay by\n\n \nStory by\n\nBong Joon Ho\n\nBong Joon Ho and Han Jin Won\n\n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n\n \n \n \n \n \n \n \n \n \n\n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n\n\x0c1\n\n2\n\nTITLE SEQUENCE OVER',
             'Scripts.com\n\nSpotlight\n\nBy Tom McCarthy\n\nPage 1/152\n\n\x0c1',
             'STRAIGHT OUTTA COMPTON\n\nBy\n\nLeigh Savidge and Alan Wenkus\n\nRevised 8/16/08',
            '\nXenon Pictures','\n(310)451-5510','\n\n\x0cEXT.',
            'Scripts.com\n\nGet Out\n\nBy Jordan Peele\n\nPage 1/94\n\n\x0c',
            'NEONratedAwards.com\n\n ',
            'THE LOBSTER\n\nWritten by',
            '\n\nYorgos Lanthimos', 
            '& Efthimis Filippou\n\n30 April 2013',
            '\n\nElement Pictures\nLimp\nScarlet', 
            'Films\n\n 1',
             'Scripts.com\n\nThe Lobster\n\nBy Yorgos Lanthimos\n\nPage 1/39\n\n\x0c',
            ' BLACK \nAccompanied by dark but curiously upbeat MUSIC. \nAt the end of the credits, the MAIN TITLE, in strange \ncalligraphy, fills the screen -- \n\n“PARASITE”\n\n1\n \nMUSIC FADES. ',
           'THE END.\n\nFOR LANCE CORPORAL ALFRED H. MENDESST BATTALION, KING’S ROYAL RIFLE CORPS\n\nWHO TOLD US THE STORIES\n\n\x0c"']

def drop_start(text, ext_list):
    new_text = text
    for match in ext_list:
        new_text = re.sub(match, '', new_text)
    return new_text

In [16]:
# apply function to do initial light clean on columns and make new fields

ACS['FirstClean'] = ACS['Script'].apply(drop_start, args = (ext_list, ))
ACS

,Title,Script,year,orig_screen_win,best_pic_win,reviewer,FirstClean
0,1917,1917\n\nWritten by\n\nSam Mendes\n&\nKrysty Wi...,2019,0,Nom,Shafer,...
1,20th_Century_Women,20th Century Women\n\nby\n\nMike Mills\n\n EXT...,2016,0,None,Shafer,OCEAN - DAY \n\nHigh overhead shot looking do...
2,BridgeofSpies,Scripts.com\n\nBridge of Spies\n\nBy Matt Char...,2015,0,Nom,Shafer,\nThe height of the Cold War. The United Stat...
3,ExMachina,\n\nEX MACHINA\n\nBy \n\nAlex Garland\n\n© Al...,2015,0,None,Shafer,© \n +44 (0)207 843 4410\n\n Music starts.\n...
5,Get_Out,Scripts.com\n\nGet Out\n\nBy Jordan Peele\n\nP...,2017,1,Nom,Shafer,"I appeal to you therefore, brothers, by the me..."
8,InsideOut,INSIDE OUT\n\noriginal story by\n\nPete Docter...,2015,0,None,Shafer,\n\nJOY (V.O.)\nDo you ever look at someone an...
9,KnivesOut,KNIVES OUT\n\nA Murder Mystery by\n\nRian John...,2019,0,None,Shafer,THROMBEY ESTATE MANOR HOUSE - DAWN\n\nThe gro...
12,Manchester_By_TheSea,MANCHESTER BY THE SEA\n\nWritten & Directed \n...,2016,1,Nom,Shafer,MANCHESTER HARBOR -- SEA. DAY.\n\nA small com...
14,Parasite,F O R Y O U R C O N S I D E R A T I O N\n\...,2019,1,Win,Shafer,BLACK \n \nAccompanied by dark but curiously...
17,Spotlight,Scripts.com\n\nSpotlight\n\nBy Tom McCarthy\n\...,2015,1,Win,Shafer,"INT. POLICE STATION, BOSTON - NIGHT, 1976 1\n..."


In [17]:
# Create function to clean unneeded pieces from script. ## These used to be all one tool called "multiclean"

def remove_contractions(text):
    text = re.sub(r"n\'t", " not", text) # handle contractions
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'m", " am", text)
    text = re.sub(r"\'d", " had", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"ain\'t", "are not", text)
    return text

def remove_page_nums(text):
    text = re.sub('\n11/26/14', '', text)
    text = re.sub('\n\n\\x0c\s+\d+\.','', text)
    text = re.sub('\nPage \d+\/\d+', '', text)
    text = re.sub('\n\/Page', '', text)
    text = re.sub('\\x0c|\\x0c ', '', text)
    text = re.sub('\\x0c\d\.', ' ', text)
    text = re.sub('\n+\\x0c\d+','', text)
    text = re.sub('\n\n\\x0c\s+\d+\.|\n\n\\x0c\s+\d+\.|\\x0c','', text)
    return text

def remove_space_digs(text):
    text = re.sub('\n\*\n', '', text)
    text = re.sub("\’", '', text) #remove possesives
    text = re.sub('^\s+','',text)
    text = re.sub('\s+\d\.', '', text)
    text = re.sub('\d\.|\d+\.', '', text)
    text = re.sub('\n\s\d+\.\n|\n*\s+\\n+\d+\.', ' ', text)
    text = re.sub('[“]', '', text)
    text = re.sub('\n\d\.| \d\.|\n \d+\.|\n\d+\.', '', text)
    text = re.sub('\n\d+|\n\d+\.| \d+\n| \d+\.', '', text)
    text = re.sub('\n\*\n', '', text)
    text = re.sub('^\n\(\)- ', '', text)
    text = re.sub('[--]', '', text)
    text = re.sub('[#]', '', text)
    text = re.sub('[©]', '', text)
    text = re.sub('[“]', '', text)
    text = re.sub('^\n\(d+\)d+\-d+ ', '', text)
    text = re.sub('\s+\d', '', text)
    text = re.sub('\(\d+\)', '', text)
    text = re.sub('\d+\n+\d+\w\n+\d|\n+\d+', '', text)
    text = re.sub('\n\d+\.', '', text)
    text = re.sub('\d', '', text)
    text = re.sub(' +', ' ', text)
    text = re.sub('\n\d\.| \d\.|\n \d+\.|\n\d+\.| \d\n|\n\d|\n\d |\n\d\n', '', text)
    return text

def remove_words(text):
    text = re.sub('TITLES:', '', text)
    text = re.sub('\(V\.O\.\)|\(O\.S\.\)|\(O\.C\.\)| V.O.|O\.S\.|V\.O\.|\sV\.O\.', '', text)
    text = re.sub('^Films\n+ \d\n\n', '', text)
    text = re.sub("\(CONTINUED:\)|\(CONTINUED\)|CONTINUED:|COUNTINUOUS| CONTINUOUS|CONT'D|\(CONT'D\)", ' ', text)
    text = re.sub("\(CONT\’D\)| \(CONT\'D\)\s+|\(CONT\'D\)|\(CONT\'D\)|\s\(CONT\'D\)\s+| (CONT\’D)",'',text)
    text = re.sub("\(CONTINUED:\)|\s\(CONT\’D\)\n| \(CONT\’D\) \n|\(CONTINUED\)|CONTINUED:|COUNTINUOUS| CONTINUOUS|CONT'D|\(CONT'D\)| \(CONT'D\)|\(CONT..\d\;D\)", ' ', text)
    text = re.sub(r'14 FINAL SHOOTING SCRIPT  |FINAL SHOOTING SCRIPT|\n 12.17.14 FINAL SHOOTING SCRIPT \d+\.|\n 12.17.14 FINAL SHOOTING SCRIPT |\n+ \d+ FINAL SHOOTING SCRIPT  \n+', '', text)
    text = re.sub("\(CONT’D\)| \(CONT'D\)",'', text)
    text = re.sub("\(CONT’D\)| \(CONT'D\)| \(CONT\\'D\)",' ', text)
    text = re.sub('\(MORE\)', '', text)
    return text
    

In [18]:
# Apply cleaning functions to initially cleaned col (FirstClean)

ACS['SecondClean'] = ACS['FirstClean'].apply(remove_contractions) # does not seem to be working
ACS['SecondClean'] = ACS['SecondClean'].apply(remove_page_nums)
ACS['SecondClean'] = ACS['SecondClean'].apply(remove_space_digs)
ACS['SecondClean'] = ACS['SecondClean'].apply(remove_words)

In [19]:
# Make a combined Visual Cues and Speaking Clean column

stop_words = set(stopwords.words('english'))
punc = string.punctuation


def normalize(text):
    text = text.lower() #lowercase all text
    text = re.sub('\n', ' ', text) # remove newlines
    text = re.sub('[{):?+,<>.!?/(;}]', '', text) # remove punctuation
    text = re.sub('\[\]', '', text)
    text = re.sub("\’", '', text)
    text = re.sub('[“]', '', text) # remove various formats of quoatation marks
    text = re.sub('[”]', '', text)
    text = re.sub("\-", ' ', text)
    text = re.sub("[‘]", ' ', text)
    text = re.sub('\*', '', text)
    text = re.sub('[#]', '', text)
    text = re.sub('[©]', '', text)
    text = re.sub('\s+', ' ', text) # remove multiple spaces
    text = re.sub(r"n\'t", " not", text) # expand contractions
    text = re.sub(r"\'s", " is", text)
    text = re.sub("\'m", " am", text)
    text = re.sub(r"\'d", " had", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"ain\'t", "are not", text) 
    text = w_tokenize(text) # tokenize text
    text = [lemma.lemmatize(w) for w in text] # lemmatize text
    text = [w for w in text if w not in stop_words] # remove stop words
    return text

In [20]:
# Create column of cleaned/normalized visual cues and speaking parts combined. 

ACS['Everything_Clean'] = ACS['SecondClean'].apply(normalize)

In [21]:
# Create function to idenitfy speaking vs. visual cues; returns as what is considered as "dialogue"
## Allie's function
## used to as initial separator of visual cues from speaking parts.

def speaking_parts(text):
    # split on combo of caps and new lines
    split_text = re.split("\\n[A-Z]*\s?\\n", text)

    visual_cues = []
    # if begins with INT or EXT then it's a setting and the next line is visual cue
    settings = [v for v in range(len(split_text)) if split_text[v].startswith("INT")
                or split_text[v].startswith("EXT")]

    # extract visual cues
    visual_cue = [split_text[s + 1] for s in settings]

    # remove word if uppercase
    visual_cue = [w for w in visual_cue if w.isupper() == False]

    visual_cues.append(visual_cue)

    sets_cues_id = []
    for s in settings:
        sets_cues_id.append(s)
        sets_cues_id.append(s + 1)

    # remove settings and visual cues from dialogue
    dialogue = [split_text[d] for d in range(len(split_text)) if d not in sets_cues_id]

    # removing entries that are just all caps (i.e. CHARLIE)
    dialogue = [d for d in dialogue if d.isupper() == False]

    # removing entries that don't have a character indicator
    dialogue_wo_indicator = [t for t in dialogue if t[:3].isupper() == False]
    # split out indicators from dialogue + flatten nested list
    dialogue_split = [re.split("[A-Z]\s?\\n", t) for t in dialogue if t[:3].isupper()]
    # drop indicator + flatten split list
    dialogue_flatten = []
    for ts in dialogue_split:
        ts.pop(0)
        dialogue_flatten.append(ts)
    dialogue_flatten = [item for sublist in dialogue_flatten for item in sublist]
    dialogue = dialogue_flatten + dialogue_wo_indicator
    dialogue = w_tokenize(" ".join(dialogue))

    return dialogue

In [22]:
# Apply function to create Dialogue column for further cleaning separation
# Should result in a list of strings for each script

ACS['Diag_Work'] = ACS['SecondClean'].apply(speaking_parts)

In [23]:
# apply AR cleaning function to standardize all
ACS['cleaned_dialogue'] = ACS['Diag_Work'].apply(lambda x: " ".join(clean_lists(x)))
ACS['cleaned_script'] = ACS['Everything_Clean'].apply(lambda x: " ".join(clean_lists(x)))

# drop extra columns from cleaning
ACS.drop(['FirstClean', 'SecondClean', 'Everything_Clean', 'Diag_Work'], axis = 1, inplace=True)

In [24]:
ACS.head()

,Title,Script,year,orig_screen_win,best_pic_win,reviewer,cleaned_dialogue,cleaned_script
0,1917,1917\n\nWritten by\n\nSam Mendes\n&\nKrysty Wi...,2019,0,Nom,Shafer,find ha push right well ill make sure relay di...,ext meadow day aprilth rolling landscape rustl...
1,20th_Century_Women,20th Century Women\n\nby\n\nMike Mills\n\n EXT...,2016,0,None,Shafer,newwavepunk jog shopping cart toward commotion...,ocean day high overhead shot looking pacific o...
2,BridgeofSpies,Scripts.com\n\nBridge of Spies\n\nBy Matt Char...,2015,0,Nom,Shafer,agent watch abel train stop broad street agent...,height cold war united state soviet union fear...
3,ExMachina,\n\nEX MACHINA\n\nBy \n\nAlex Garland\n\n© Al...,2015,0,None,Shafer,monitor subject hidden bay facility hell oppor...,music start open int office day computer monit...
5,Get_Out,Scripts.com\n\nGet Out\n\nBy Jordan Peele\n\nP...,2017,1,Nom,Shafer,andre africanamerican man run sidewalk sweat l...,appeal therefore brother mercy god topresent b...


### Concatenate AR + AS

In [25]:
ar.columns

Index(['Title', 'Script', 'year', 'orig_screen_win', 'best_pic_win',
       'reviewer', 'cleaned_dialogue', 'cleaned_script'],
      dtype='object')

In [26]:
ACS.columns

Index(['Title', 'Script', 'year', 'orig_screen_win', 'best_pic_win',
       'reviewer', 'cleaned_dialogue', 'cleaned_script'],
      dtype='object')

In [27]:
# sanity check -- expect True
ar.columns == ACS.columns

array([ True,  True,  True,  True,  True,  True,  True,  True])

In [30]:
movie_df = pd.concat([ar, ACS]).reset_index(drop=True)
movie_df

,Title,Script,year,orig_screen_win,best_pic_win,reviewer,cleaned_dialogue,cleaned_script
0,First_Reformed,Scripts.com\n\nFirst Reformed\n\nBy Paul Schra...,2018,0,None,Ragan,keep journal word program digital file longhan...,bird calling distance toller keep journal word...
1,GreenBook,GREEN BOOK \n\nWritten by \n\nNick Vallelonga ...,2018,1,Win,Ragan,thank much welcome jules podell copacabana bob...,fade title card new york city 1962 ext copacab...
2,Hell_or_HighWater,Scripts.com\n\nHell or High Water\n\nBy Taylor...,2016,0,Nom,Ragan,quiet open door open door devil please stand t...,quiet open door open door devil please stand t...
3,LadyBird,\n \n\n \n \n \n \n \n \n \n \n \n \n \n \n \...,2017,0,Nom,Ragan,making bed looking back loan scholarship world...,anybody talk california hedonism ha never spe...
4,LaLaLand,LA LA LAND\n\nby\n\nDamien Chazelle\n\n A sun-...,2016,0,Nom,Ragan,think going worst working like gon na talk tal...,sunblasted sky hear radio one piece music ano...
5,MarriageStory,MARRIAGE STORY\n\nWritten and Directed by Noah...,2019,0,Nom,Ragan,make people feel comfortable even embarrassing...,charlie love nicole int theater manhattan day ...
6,Roma,IN ENGLISH\n\n ROMA\n\nWritten and Directed by...,2018,0,Nom,Ragan,said bed look mommy grade watermelon cleo coul...,int patio tepeji 21 day yellow triangle inside...
7,ShapeofWater,F O R Y O U R C O N S I D E R A T I O N\n\...,2017,0,Win,Ragan,spoke it did would tell wonder would tell time...,int elisa apartment river footage night float ...
8,TheBigSick,THE BIG SICK\n\nby\nEmily V. Gordon & Kumail N...,2017,0,None,Ragan,bob dalaban true cokehead gon na say anything ...,black hear audience laughter andy dodd give ma...
9,TheFavourite,F O R Y O U R C O N S I D E R A T I O N\n\...,2018,0,Nom,Ragan,mud stink lady marlborough yes like lisp love ...,ext countryside carriage cross landscape int c...
